# [실습1] LangGraph

## 실습 목표
---
엑셀 데이터 분석과 RAG 기능을 모두 활용하기 위해, 그래프 구조의 챗봇을 구성할 수 있는 LangGraph의 기초를 학습합니다.

## 실습 목차
---

1. **LangGraph:** 체인을 넘어서 그래프 구조를 구성하기 위해 LangChain의 기초를 학습하고, 1챕터에서 구현한 간단한 LLM 챗봇을 다시 한번 구성합니다.

## 실습 개요
---
그래프 구조의 챗봇을 구성할 수 있는 LangGraph를 사용해 봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [8]:
from typing import Annotated

from IPython.display import Image, display
from langchain_community.chat_models import ChatOllama
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict

- Ollama를 통해 Mistral 7B 모델을 불러옵니다. 처음 불러오는데는 다운로드 시간 포함 약 3분이 소요됩니다.

In [ ]:
!ollama pull mistral:7b

## 1. LangGraph
- 체인을 넘어서 그래프 구조를 구성하기 위해 LangChain의 기초를 학습합니다.

먼저, mistral:7b 모델을 사용하는 ChatOllama 객체를 생성합니다.

In [9]:
llm = ChatOllama(model="mistral:7b")
tool_llm = ChatOllama(model="mistral:7b") 

이번 실습에서는 매우 간단한 챗봇을 하나 만들어 볼 것입니다. LLM에 질문을 입력하면 LLM은 별다른 액션을 취하지 않고 바로 질문에 대답할 것입니다.

앞서 체인을 구성할 때는 | 연산자를 통해 Agent, Tool 등 여러 구성 요소를 엮어 하나의 체인으로 만들었습니다.

LangGraph의 그래프를 구성할 때는 `StateGraph` 객체를 생성하고, 여기에 `Node` (노드) 와 `Edge` (엣지, 간선)를 추가하여 하나의 그래프로 만들 수 있습니다.

In [10]:
# TypedDict는 딕셔너리와 비슷한 구조를 가진 클래스로, 각 키와 값의 타입을 미리 정의할 수 있습니다.

class State(TypedDict):

    messages: Annotated[list, add_messages]



graph_builder = StateGraph(State)

1. class State(TypedDict):
State라는 클래스 생성

2. TypedDict
TypedDict는 딕셔너리와 비슷한 구조를 가진 클래스로, 
각 키와 값의 타입을 미리 정의할 수 있습니다.

3. messages: Annotated[list, add_messages]
messages의 데이터타입은 list이고 add_messages라는 메타 데이터를 갖습니다.

- 메타 데이터 
'데이터에 대한 데이터'를 의미합니다.
어떤 데이터에 대한 추가적인 정보를 제공해주는 데이터입니다.

ex)이미지에 대한 메타 데이터
촬영일시: 2024-10-16
장소: 서울
해상도: 1920x1080
파일 크기: 2MB

4. graph_builder = StateGraph(State)
StateGraph는 상태(노드)를 그래프로 관리하는 객체입니다.
StateGraph(State)는 이 그래프가 State 구조를 기반으로 동작하도록 설정합니다.

State 클래스는 기차역을 관리하는 중앙 제어 시스템입니다.
messages는 역에 도착한 메시지 목록이고, add_messages는 그 메시지를 정리하거나 더하는 작업자입니다.
graph_builder는 이 모든 것을 노선(그래프)으로 연결해주는 설계자입니다.



그래프는 Node와 Edge로 구성되어 있습니다.

Node는 체인의 각 구성 요소에 대응합니다. Agent, Tool, LLM 등 그래프의 각 구성 요소를 의미합니다.

Edge는 Node를 연결하는 요소로, Node에서 정보를 어느 Node로 전달해야 하는지를 나타냅니다.
- 체인은 일렬로 이어져 있기 때문에, 사용자의 입력을 연결된 순서대로 통과시키면 원하는 결과를 얻을 수 있습니다.
- 하지만, 그래프는 일렬로 이어져 있지 않기 때문에 Edge를 통해 정보를 전달하는 순서 및 방향을 정해줘야 합니다.

여기서는 간단한 챗봇을 만들기 위해 (시작 지점과 끝점을 제외한) 단일 노드 그래프를 만들어 보겠습니다.

In [ ]:
## Node 생성
# Node는 그래프에서 실행될 수 있는 작업을 정의합니다.
# Node는 함수로 정의되며, StateGraph를 정의할 때 사용한 State type을 입력으로 받습니다.
# Node는 state를 업데이트하거나, 새로운 state를 반환할 수 있습니다.
def answer(state: State):
    # 단순 챗봇이므로, 입력받은 메시지를 LLM에 입력하고, 그 결과를 반환합니다.
    return {"messages": [llm.invoke(state["messages"])]}

## Node 추가
# 그래프에 Node를 추가합니다.
# add_node 함수의 첫 번째 인자는 Node의 이름입니다. (함수 이름과 달라도 됩니다)
# Node의 이름은 특정 노드에서 다른 노드를 호출할 때 사용합니다.
# 두 번째 인자는 Node가 호출될 때 실행될 함수입니다.
graph_builder.add_node("answer", answer)


# 그래프를 실행할 때 시작점이 되는 Node를 설정합니다. (Node 이름을 전달합니다)
# 이 경우, 사용자의 입력을 받아 `answer` 노드를 가장 먼저 실행하게 됩니다.
graph_builder.set_entry_point("answer")

# 그래프를 실행할 때 종료점이 되는 Node를 설정합니다. (Node 이름을 전달합니다)
# 이 경우, `answer` 노드가 실행된 후 사용자에게 결과를 반환하고 그래프를 종결합니다.
graph_builder.set_finish_point("answer")


1. def answer(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

answer 함수
메시지를 받아 응답을 생성하는 함수입니다.

매개변수
state: State 클래스 인스턴스로, 메시지들을 포함하고 있습니다.

{"messages": [llm.invoke(state["messages"])]}
state["messages"]는 state 안에 있는 메시지 리스트를 가져옵니다.
llm.invoke(): 이 함수는 대형 언어 모델(LLM)을 호출해, 메시지 리스트를 처리하고 응답을 생성합니다.
최종적으로, 언어 모델의 응답을 담은 새로운 메시지 리스트를 반환합니다.

2. graph_builder.add_node("answer", answer)

add_node(): 그래프에 answer라는 이름으로 노드를 추가합니다.

이 노드는 메시지를 처리하고 응답을 반환하는 역할을 담당합니다. 
즉, answer 함수가 하나의 노드처럼 등록되는 것입니다.

3. graph_builder.set_entry_point("answer")


set_entry_point(): 이 코드는 그래프가 시작되는 지점(첫 번째 노드)을 설정합니다.
여기서는 answer라는 노드(함수)가 시작점이므로, 처음에 answer 함수가 실행됩니다.

4. graph_builder.set_finish_point("answer")

set_finish_point(): 이 코드는 그래프가 끝나는 지점(마지막 노드)을 설정합니다.
여기서도 answer가 마지막에 실행되도록 설정되었습니다.
즉, 시작과 끝이 모두 answer이므로, 메시지를 처리하는 흐름이 단순하게 한 번의 처리가 끝나면 종료되는 구조입니다.




graph_builder.add_node("answer", answer) 같은 노드 추가 메서드에서 두 입력값의 차이는 아래와 같습니다.

첫 번째 인자 ("answer"):
노드의 이름 또는 레이블입니다.

두 번째 인자 (answer):
노드에 포함되는 실제 데이터입니다.

예시
graph_builder.add_node("question", "What is LangChain?")
graph_builder.add_node("answer", "LangChain is a framework for building applications with LLMs.")

"question"과 "answer"는 노드의 이름입니다.
"What is LangChain?"과 "LangChain is a framework...는 각 노드에 포함되는 데이터입니다.

그래프 구성을 종료했으면, 이제 그래프를 컴파일하여 사용할 수 있습니다.

컴파일 완료된 그래프는 "Runnable" 한 객체입니다.<br>
즉, `invoke()` 메서드를 통해 그래프를 사용할 수 있고, 앞서 배운 체인의 구성 요소로 사용할 수 있습니다.

In [12]:
# graph_builder: LangChain에서 사용되는 그래프 빌더 객체입니다. 
# 이 객체는 서로 연결된 여러 컴포넌트(노드)들을 정의해 하나의 작업 흐름(그래프)을 구축합니다.

# compile(): 그래프 빌더가 작성한 그래프를 실제로 컴파일하여 사용 가능한 그래프 객체로 변환합니다. 
# 이 과정에서 모든 노드 간의 연결 및 데이터 흐름이 유효한지 검증됩니다.

# 결과: 이 코드가 실행되면 graph라는 변수에 컴파일된 그래프가 저장됩니다. 
# 이제 이 그래프를 시각화하거나 실행할 수 있는 상태가 됩니다.

graph = graph_builder.compile()

컴파일된 그래프의 구조를 시각화 해봅시다.

단일 노드 그래프이기 떄문에 시작점, 끝점을 제외하면 노드 간의 Edge가 없는 것을 확인할 수 있습니다.

In [ ]:
# graph.get_graph(): 컴파일된 그래프에서 시각화 가능한 그래프 객체를 가져옵니다.

# draw_mermaid_png(): Mermaid.js 형식으로 이 그래프를 PNG 이미지로 변환합니다.

# Mermaid는 그래프와 다이어그램을 텍스트 기반으로 그릴 수 있는 툴입니다.

# 이 명령은 Mermaid 형식으로 내부적으로 렌더된 그래프를 PNG 이미지로 변환합니다.
# Image(): 파이썬의 IPython.display 모듈에서 제공하는 이미지 객체로, PNG 이미지를 화면에 표시할 수 있도록 만듭니다.

# display(): IPython의 display() 함수로 이미지 객체를 노트북이나 웹 애플리케이션에 렌더링합니다.

display(Image(graph.get_graph().draw_mermaid_png()))

이제 챗봇을 사용해봅시다. 
- 시스템 프롬프트에 별도로 한글 텍스트를 추가하지 않았고, 한글로 답변하라는 프롬프트도 없기 때문에 대부분 영어로 답변합니다.

In [ ]:
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        # graph.stream 함수를 사용하여 그래프를 실행하고, 각 Node의 결과를 반환합니다.
        for event in graph.stream({"messages": ("user", question)}):
            # 각 Node의 결과를 출력합니다.
            for value in event.values():
                print("Assistant:", value["messages"][-1].content)

In [ ]:
# 아래 코드의 주석을 해제하고 실행하면 본 실습에서 다운받은 모델 파일을 삭제합니다.
# 각 실습에서 같은 모델이라도 다시 다운 받기 때문에, 
# 실습이 종료되었으면 아래 명령어를 실행하여 불필요한 파일을 삭제하는 것이 좋습니다.
!rm -rf .ollama/models/*